In [35]:
import scipy.io as sio
from scipy.misc import imread
from preprocess.normalize import preprocess_signature
import tf_signet
from tf_cnn_model import TF_CNNModel
import tensorflow as tf
import numpy as np
import pandas as pd
import sys
import os
import scipy.io
from find_largest_image import find_largest
import tqdm 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
import random
from numpy.random import choice

In [36]:
data_folder = 'C:\\Users\\Mert\\Documents\\GitHub\\sigver_bmg\\data\\downloaded_pp_features\\gpds_signet_f_all'

In [37]:
datalist = [x for x in os.listdir(data_folder) if not x.endswith('.csv')]

In [38]:
rc=0 # row counter
for data in tqdm.tqdm(datalist, ascii=True):
    # print(fakeness)
    user_id = int(data.split('.')[0].split('_')[-1])
    matfile = sio.loadmat(os.path.join(data_folder, data))
    for row in range(matfile['features'].shape[0]):
        rc+=1
print("The .csv should contain", rc, 'rows!')




  0%|                                                                                         | 0/1762 [00:00<?, ?it/s]


 16%|############6                                                                | 290/1762 [00:00<00:00, 2878.96it/s]


 33%|#########################                                                    | 574/1762 [00:00<00:00, 2865.11it/s]


 50%|######################################7                                      | 886/1762 [00:00<00:00, 2935.12it/s]


 64%|################################################6                           | 1127/1762 [00:00<00:00, 2748.14it/s]


 80%|############################################################9               | 1414/1762 [00:00<00:00, 2781.67it/s]


 98%|##########################################################################1 | 1719/1762 [00:00<00:00, 2851.22it/s]


100%|############################################################################| 1762/1762 [00:00<00:00, 2844.83it/s]

The .csv should contain 47458 rows!


In [39]:
data_f = pd.DataFrame(np.zeros((rc,3)),columns=['user_id','sig_id','fakeness'])
visual_f = pd.DataFrame(np.zeros((rc,2048)))

In [40]:
rc=0 # row counter
for data in tqdm.tqdm(datalist, ascii=True):
    if not(data.endswith('.mat')): continue
    if (data.split('.')[0].split('_')[0] == 'forg'): fakeness = 1
    else: fakeness = 0
    # print(fakeness)
    user_id = int(data.split('.')[0].split('_')[-1])
    matfile = sio.loadmat(os.path.join(data_folder, data))
    for row in range(matfile['features'].shape[0]):
        data_f.iloc[rc]['user_id'] = user_id
        data_f.iloc[rc]['sig_id'] = int(row+1)
        data_f.iloc[rc]['fakeness'] = fakeness
        visual_f.iloc[rc] = np.expand_dims(matfile['features'][row], 1).T
        rc+=1




  0%|                                                                                         | 0/1762 [00:00<?, ?it/s]


  0%|2                                                                                | 6/1762 [00:00<00:34, 51.42it/s]


  1%|5                                                                               | 12/1762 [00:00<00:32, 53.55it/s]


  1%|8                                                                               | 18/1762 [00:00<00:32, 53.60it/s]


  1%|#1                                                                              | 25/1762 [00:00<00:30, 57.05it/s]


  2%|#4                                                                              | 32/1762 [00:00<00:29, 59.22it/s]


  2%|#7                                                                              | 38/1762 [00:00<00:29, 59.41it/s]


  2%|#9                                                                              | 44/1762 [00:00<00:29, 57.91it/s]


  3%|##3             

 25%|###################9                                                           | 446/1762 [00:07<00:23, 57.03it/s]


 26%|####################3                                                          | 453/1762 [00:07<00:22, 58.98it/s]


 26%|####################5                                                          | 459/1762 [00:07<00:21, 59.24it/s]


 26%|####################8                                                          | 465/1762 [00:07<00:22, 57.30it/s]


 27%|#####################1                                                         | 471/1762 [00:07<00:23, 55.94it/s]


 27%|#####################3                                                         | 477/1762 [00:08<00:22, 56.18it/s]


 27%|#####################6                                                         | 483/1762 [00:08<00:22, 56.59it/s]


 28%|#####################9                                                         | 489/1762 [00:08<00:22, 56.64it/s]


 28%|###################

 50%|#######################################7                                       | 887/1762 [00:15<00:14, 61.84it/s]


 51%|########################################1                                      | 896/1762 [00:15<00:12, 67.07it/s]


 51%|########################################5                                      | 905/1762 [00:15<00:12, 70.70it/s]


 52%|########################################9                                      | 913/1762 [00:15<00:12, 70.32it/s]


 52%|#########################################2                                     | 921/1762 [00:15<00:11, 70.89it/s]


 53%|#########################################6                                     | 929/1762 [00:15<00:11, 73.15it/s]


 53%|##########################################                                     | 937/1762 [00:15<00:11, 73.69it/s]


 54%|##########################################4                                    | 946/1762 [00:15<00:10, 75.72it/s]


 54%|###################

 83%|################################################################4             | 1457/1762 [00:22<00:04, 75.64it/s]


 83%|################################################################8             | 1465/1762 [00:22<00:03, 76.08it/s]


 84%|#################################################################2            | 1473/1762 [00:22<00:03, 76.39it/s]


 84%|#################################################################5            | 1481/1762 [00:22<00:03, 75.64it/s]


 85%|#################################################################9            | 1489/1762 [00:22<00:03, 75.43it/s]


 85%|##################################################################2           | 1497/1762 [00:22<00:03, 76.70it/s]


 85%|##################################################################6           | 1505/1762 [00:23<00:03, 76.27it/s]


 86%|##################################################################9           | 1513/1762 [00:23<00:03, 73.38it/s]


 86%|###################

In [41]:
# data_f = data_f.reset_index().drop(columns=['index'])
# visual_f = visual_f.reset_index().drop(columns=['index'])

In [42]:
visual_f.shape

(47458, 2048)

In [43]:
data_f.shape

(47458, 3)

In [44]:
data_f.to_csv(os.path.join(data_folder,'data_features.csv'), index=False)
visual_f.to_csv(os.path.join(data_folder,'visual_features.csv'), index=False)